#Telecom Domain ReadOps Assignment
This notebook contains assignments to practice Spark read options and Databricks volumes. <br>
Sections: Sample data creation, Catalog & Volume creation, Copying data into Volumes, Path glob/recursive reads, toDF() column renaming variants, inferSchema/header/separator experiments, and exercises.<br>

![](https://fplogoimages.withfloats.com/actual/68009c3a43430aff8a30419d.png)
![](https://theciotimes.com/wp-content/uploads/2021/03/TELECOM1.jpg)

##First Import all required libraries & Create spark session object

##1. Write SQL statements to create:
1. A catalog named telecom_catalog_assign
2. A schema landing_zone
3. A volume landing_vol
4. Using dbutils.fs.mkdirs, create folders:<br>
/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/
/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/
/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/
5. Explain the difference between (Just google and understand why we are going for volume concept for prod ready systems):<br>
a. Volume vs DBFS/FileStore<br>
b. Why production teams prefer Volumes for regulated data<br>

In [0]:
%sql
--create catalog if not exists telecom_catalog_assign;
create schema if not exists telecom_catalog_assign.landing_zone;
create volume if not exists telecom_catalog_assign.landing_zone.landing_vol;

In [0]:
base_path = "/Volumes/telecom_catalog_assign/landing_zone/landing_vol/"
cust_path = dbutils.fs.mkdirs(base_path + "customer/")
usage_Path = dbutils.fs.mkdirs(base_path + "usage/")
tower_path = dbutils.fs.mkdirs(base_path + "tower/")


In [0]:
#this data was given 
##Data files to use in this usecase:
customer_csv = '''
101,Arun,31,Chennai,PREPAID
102,Meera,45,Bangalore,POSTPAID
103,Irfan,29,Hyderabad,PREPAID
104,Raj,52,Mumbai,POSTPAID
105,,27,Delhi,PREPAID
106,Sneha,abc,Pune,PREPAID
'''

usage_tsv = '''customer_id\tvoice_mins\tdata_mb\tsms_count
101\t320\t1500\t20
102\t120\t4000\t5
103\t540\t600\t52
104\t45\t200\t2
105\t0\t0\t0
'''

tower_logs_region1 = '''event_id|customer_id|tower_id|signal_strength|timestamp
5001|101|TWR01|-80|2025-01-10 10:21:54
5004|104|TWR05|-75|2025-01-10 11:01:12
'''
tower_logs_region2 = '''event_id|customer_id|tower_id|signal_strength|timestamp
5002|102|TWR02|-85|2025-01-10 10:21:54
5003|103|TWR03|-80|2025-01-10 10:21:54
5005|105|TWR04|-70|2025-01-10 11:01:12'''

##2. Filesystem operations
1. Write code to copy the above datasets into your created Volume folders:
Customer → /Volumes/.../customer/
Usage → /Volumes/.../usage/
Tower (region-based) → /Volumes/.../tower/region1/ and /Volumes/.../tower/region2/

2. Write a command to validate whether files were successfully copied

In [0]:
dbutils.fs.put("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer_csv",customer_csv,overwrite=True)
dbutils.fs.put("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage_tsv",usage_tsv,overwrite=True)
dbutils.fs.put("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower_logs_r1",tower_logs_region1,overwrite=True)
dbutils.fs.put("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower_logs_r2",tower_logs_region2,overwrite=True)

dbutils.fs.ls("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/")

In [0]:
from pyspark.sql.session import SparkSession
print(spark)
spark1=SparkSession.builder.getOrCreate()
print(spark1)

##3. Directory Read Use Cases
1. Read all tower logs using:
Path glob filter (example: *.csv)
Multiple paths input
Recursive lookup

2. Demonstrate these 3 reads separately:
Using pathGlobFilter
Using list of paths in spark.read.csv([path1, path2])
Using .option("recursiveFileLookup","true")

3. Compare the outputs and understand when each should be used.

In [0]:
tower_df1 = spark.read.csv(path=["/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/"],header=True,inferSchema=True,sep="|",pathGlobFilter="tower_logs_*",recursiveFileLookup=True)
print(tower_df1.count())
display(tower_df1)
usage_df1 = spark.read.csv(path=["/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/"],header=True,inferSchema=True,sep="\t",
pathGlobFilter="usage*",recursiveFileLookup=True)
print(usage_df1.count())
display(usage_df1)
cust_df1 = spark.read.csv(path=["/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/"],pathGlobFilter="*sv",recursiveFileLookup=True,inferSchema=True,header=True,sep=",").toDF("customer_id","name","age","city","plan_type")
print(cust_df1.count())
display(cust_df1)

##4. Schema Inference, Header, and Separator
1. Try the Customer, Usage files with the option and options using read.csv and format function:<br>
header=false, inferSchema=false<br>
or<br>
header=true, inferSchema=true<br>
2. Write a note on What changed when we use header or inferSchema  with true/false?<br>
3. How schema inference handled “abc” in age?<br>

In [0]:
#1&2 are covered with true/false in header/inferSchema
cust_df2=spark.read.option("header","false").option("inferSchema","true").option("sep",",").format("csv").load("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer_csv").toDF("customer_id","name","age","city","plan_type")
cust_df2.printSchema() 
cust_df2.show(2)
usage_df2=spark.read.option("header","true").option("inferSchema","true").option("sep","\t").format("csv").load("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage_tsv")
usage_df2.printSchema()
usage_df2.show(2)
#3 the age column has 'abc' which is string value, hence entire column is assumed as string when inferschema is set to true, id inferschema is set as false, by default schema is set as string for all columns.

##5. Column Renaming Usecases
1. Apply column names using string using toDF function for customer data
2. Apply column names and datatype using the schema function for usage data
3. Apply column names and datatype using the StructType with IntegerType, StringType, TimestampType and other classes for towers data 

In [0]:
from pyspark.sql.types import StructType,StructField,IntegerType,StringType,TimestampType
cust_schema = StructType([StructField("customer_id",IntegerType(),False),
                            StructField("name",StringType(),True),
                            StructField("age",IntegerType(),True),
                            StructField("city",StringType(),True),
                            StructField("Plan_type",StringType(),True)])
customer_str_struct = "cust_id integer,name string,age integer,city string, plan_type string"
tower_schema = StructType([StructField("event_id",IntegerType(),False),
                            StructField("customer_id",IntegerType(),True),
                            StructField("tower_id",StringType(),True),
                            StructField("signal_strength",IntegerType(),True),
                            StructField("timestamp",TimestampType(),True)])
#1.
cust_df3=spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer_csv",header=True,inferSchema=True).toDF("customer_id","name","age","city","plan_type")
cust_df3.printSchema() 
cust_df3.show(2)
#2.
cust_df4 = spark.read.schema(customer_str_struct).csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer_csv")
cust_df4.printSchema()
cust_df4.show(2)
#3.
tower_df2 = spark.read.schema(tower_schema).csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/tower*")
tower_df2.printSchema()
tower_df2.show(2)
cust_df5 = spark.read.schema(cust_schema).csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer_csv")
cust_df3.printSchema()
cust_df3.show(2)


## 6. More to come (stay motivated)....